# K-means Algorithm

In [8]:
import numpy as np

In [9]:
import pandas as pd
def to_clipboard(text):
    df = pd.DataFrame([text])
    df.to_clipboard(index=False,header=False)

In [10]:
class Point:
    def __init__(self, coords = []):
        self.coords = tuple(coords)
    def euc_dist_from(self, point) -> float:
        return np.linalg.norm(np.array(point.coords) - np.array(self.coords))
    def add(self, point):
        self.coords = tuple(np.array(self.coords) + np.array(point.coords))
    def div(self, denum: int):
        self.coords = tuple(np.array(self.coords)/denum)
    def __str__(self):
        return str(self.coords)
    def __len__(self):
        return len(self.coords)
    def __hash__(self):
        return hash(self.coords)
    def __round__(self, num_dec):
        return tuple(np.round(np.array(self.coords), num_dec))

In [11]:
class Kmeans:
    def __init__(self, data = [[]], centroids = [[]]):
        self.data = [Point(item) for item in data]
        self.centroids = [Point(item) for item in centroids]
        self.init_clusters()

    def init_clusters(self):
        self.clusters = {f'Cluster-{idx+1}': [] for idx in range(len(self.centroids))}

    def __str__(self):
        return f'data: {data}\centroids: {centroids}'

    def kmeans_process(self):
        prev_clusters = None
        while True:
            self.init_clusters()
            # find each point's cluster
            for point in self.data:
                # calculating the distances from each cluster's centroid
                dists = []
                for centroid in self.centroids: dists.append(point.euc_dist_from(centroid))
                # zero-base index of min distance
                winnig_idx = np.argmin(np.array(dists))
                # recording the clustering
                self.clusters[f'Cluster-{winnig_idx+1}'].append(point)    
            # check for termination croiteria
            if prev_clusters == self.clusters: break
            # prepare next loop
            prev_clusters = self.clusters.copy()
            self.update_centroids(self.clusters)

    def update_centroids(self, clusters):
        for idx, points in enumerate(clusters.values()):
            new_centroid = Point([0]*len(self.data[0]))
            # calculating the new centroid
            for point in points: new_centroid.add(point)
            new_centroid.div(len(points))
            # saveing the new centroid
            self.centroids[idx] = new_centroid

    def print_kmeans_process(self):
        prev_clusters, prev_centroids = {}, []
        rcnt = 1
        text = []
        while True:
            # text = []
            rnum = 3
            self.init_clusters()
            text.append(f"Iteration-{rcnt}:\n")
            # find each point's cluster
            for point in self.data:
                tnum, tab = 1, '\t'
                text.append(f"{tab*tnum}Determining the cluster of the point {point}:\n")
                # calculating the distances from each cluster's centroid
                tnum+=1
                dists = []
                for centroid in self.centroids:
                    text.append(f"{tab*tnum}Distance from the centroid {round(centroid, rnum)}: ")
                    sub_text = []
                    for item in zip(point.coords, centroid.coords): sub_text.append(f"({round(item[0], rnum)} - {round(item[1], rnum)})\u00b2")
                    # text.append(u"\u221a")
                    text.append(f"\u221a[{' + '.join(sub_text)}] = {round(point.euc_dist_from(centroid), rnum)}\n")
                    dists.append(point.euc_dist_from(centroid))
                # zero-base index of min distance
                winnig_idx = np.argmin(np.array(dists))
                # recording the clustering
                self.clusters[f'Cluster-{winnig_idx+1}'].append(point)    
                text.append(f"\n{tab*tnum}Min distance is {round(dists[winnig_idx], 3)} => Point: {point} --> Cluster-{winnig_idx+1}\n\n")
            # check for termination croiteria
            text.append(f"{tab*tnum}Current {self.print_clusters(tnum = tnum)}\n\n")
            text.append(f"{tab*tnum}Previous {self.print_clusters(prev_clusters, prev_centroids, tnum = tnum)}\n\n")
            text.append(f"{tab*tnum}Are they equal? ")
            if prev_clusters == self.clusters: 
                text.append(f"Yes! => We stop here!\n\n")
                text.append(f"Final {self.print_clusters(tnum = 0)}\n")
                # print(''.join(text))
                break
            text.append(f"No! => We move to the next round!\n\n")
            # prepare next loop
            prev_clusters, prev_centroids = self.clusters.copy(), self.centroids.copy()
            # tnum+=1
            text.append(f"{tab*tnum}Update centroids: {self.print_update_centroids(self.clusters, tnum = tnum, rnum = rnum)}\n\n")
            rcnt += 1

            # print(''.join(text))
        return ''.join(text)

    def print_update_centroids(self, clusters, tnum = 0, rnum = 3):
        text = []
        tab = '\t'
        tnum += 1
        for idx, cluster in enumerate(clusters.items()):
            new_centroid = Point([0]*len(self.data[0]))
            # calculating the new centroid
            text.append(f"\n{tab*tnum}{cluster[0]}:\n")
            matrix = []
            for point in cluster[1]:
                new_centroid.add(point)
                matrix.append(round(point, rnum))
            new_centroid.div(len(cluster[1]))
            self.centroids[idx] = new_centroid
            
            matrix = np.transpose(matrix).tolist()
            sub_text = []
            tnum+=1
            for idx1, coord in enumerate(round(new_centroid, rnum)):
                sub_text.append(f"{tab*tnum}({' + '.join([str(item) for item in matrix[idx1]])})/{len(cluster[1])} = {coord}\n")
            text.append(f"{''.join(sub_text)}")
            # saveing the new centroid
            text.append(f"{tab*tnum}The new centroid is {round(new_centroid, rnum)}")
            tnum-=1
            
        return ''.join(text)

    def print_clusters(self, clusters = None, centroids = None, tnum = 0):
        if clusters is None: clusters = self.clusters
        if centroids is None: centroids = self.centroids
        tab = '\t'
        text = [f"Clustering Result: "]
        tnum+=1
        for cluster, centroid in zip(clusters.items(), centroids):
            text.append(f"\n{tab*tnum}{cluster[0]} with Centroid {round(centroid, 3)}:\n{tab*(tnum+1)}")
            sub_text = []
            for point in cluster[1]:
                sub_text.append(f"{point}")
            text.append(', '.join(sub_text))
        return ''.join(text) 

In [12]:
kmeans = Kmeans(
    data = [[3,2],[27,2],[20,8],[25,1],[12,4],[21,6],[18,6],[14,5],[16,5],[24,4],[21,4],[26,3],[3,5],[14,8],[1,10]],
    centroids = [[20,8],[14,5],[26,3]]
)
# kmeans.kmeans_process()
computation = kmeans.print_kmeans_process()
to_clipboard(computation)
print(computation)
# kmeans.print_clusters()

Iteration-1:
	Determining the cluster of the point (3, 2):
		Distance from the centroid (20, 8): √[(3 - 20)² + (2 - 8)²] = 18.028
		Distance from the centroid (14, 5): √[(3 - 14)² + (2 - 5)²] = 11.402
		Distance from the centroid (26, 3): √[(3 - 26)² + (2 - 3)²] = 23.022

		Min distance is 11.402 => Point: (3, 2) --> Cluster-2

	Determining the cluster of the point (27, 2):
		Distance from the centroid (20, 8): √[(27 - 20)² + (2 - 8)²] = 9.22
		Distance from the centroid (14, 5): √[(27 - 14)² + (2 - 5)²] = 13.342
		Distance from the centroid (26, 3): √[(27 - 26)² + (2 - 3)²] = 1.414

		Min distance is 1.414 => Point: (27, 2) --> Cluster-3

	Determining the cluster of the point (20, 8):
		Distance from the centroid (20, 8): √[(20 - 20)² + (8 - 8)²] = 0.0
		Distance from the centroid (14, 5): √[(20 - 14)² + (8 - 5)²] = 6.708
		Distance from the centroid (26, 3): √[(20 - 26)² + (8 - 3)²] = 7.81

		Min distance is 0.0 => Point: (20, 8) --> Cluster-1

	Determining the cluster of the point (

In [13]:
kmeans = Kmeans(
    data = [[3,2],[27,2],[20,8],[25,1],[12,4],[21,6],[18,6],[14,5],[16,5],[24,4],[21,4],[26,3],[3,5],[14,8],[1,10]],
    centroids = [[3,2],[16,5],[1,10]]
)
# kmeans.kmeans_process()
computation = kmeans.print_kmeans_process()
to_clipboard(computation)
print(computation)
# kmeans.print_clusters()

Iteration-1:
	Determining the cluster of the point (3, 2):
		Distance from the centroid (3, 2): √[(3 - 3)² + (2 - 2)²] = 0.0
		Distance from the centroid (16, 5): √[(3 - 16)² + (2 - 5)²] = 13.342
		Distance from the centroid (1, 10): √[(3 - 1)² + (2 - 10)²] = 8.246

		Min distance is 0.0 => Point: (3, 2) --> Cluster-1

	Determining the cluster of the point (27, 2):
		Distance from the centroid (3, 2): √[(27 - 3)² + (2 - 2)²] = 24.0
		Distance from the centroid (16, 5): √[(27 - 16)² + (2 - 5)²] = 11.402
		Distance from the centroid (1, 10): √[(27 - 1)² + (2 - 10)²] = 27.203

		Min distance is 11.402 => Point: (27, 2) --> Cluster-2

	Determining the cluster of the point (20, 8):
		Distance from the centroid (3, 2): √[(20 - 3)² + (8 - 2)²] = 18.028
		Distance from the centroid (16, 5): √[(20 - 16)² + (8 - 5)²] = 5.0
		Distance from the centroid (1, 10): √[(20 - 1)² + (8 - 10)²] = 19.105

		Min distance is 5.0 => Point: (20, 8) --> Cluster-2

	Determining the cluster of the point (25, 1):


In [15]:
kmeans = Kmeans(
    data = [[2,4],[3,2],[4,5],[7,7],[9,7],[9,9],[11,8],[14,7],[15,5],[17,5]],
    centroids = [[7,7],[11,8],[14,7]]
)
# kmeans.kmeans_process()
computation = kmeans.print_kmeans_process()
to_clipboard(computation)
print(computation)
# kmeans.print_clusters()

Iteration-1:
	Determining the cluster of the point (2, 4):
		Distance from the centroid (7, 7): √[(2 - 7)² + (4 - 7)²] = 5.831
		Distance from the centroid (11, 8): √[(2 - 11)² + (4 - 8)²] = 9.849
		Distance from the centroid (14, 7): √[(2 - 14)² + (4 - 7)²] = 12.369

		Min distance is 5.831 => Point: (2, 4) --> Cluster-1

	Determining the cluster of the point (3, 2):
		Distance from the centroid (7, 7): √[(3 - 7)² + (2 - 7)²] = 6.403
		Distance from the centroid (11, 8): √[(3 - 11)² + (2 - 8)²] = 10.0
		Distance from the centroid (14, 7): √[(3 - 14)² + (2 - 7)²] = 12.083

		Min distance is 6.403 => Point: (3, 2) --> Cluster-1

	Determining the cluster of the point (4, 5):
		Distance from the centroid (7, 7): √[(4 - 7)² + (5 - 7)²] = 3.606
		Distance from the centroid (11, 8): √[(4 - 11)² + (5 - 8)²] = 7.616
		Distance from the centroid (14, 7): √[(4 - 14)² + (5 - 7)²] = 10.198

		Min distance is 3.606 => Point: (4, 5) --> Cluster-1

	Determining the cluster of the point (7, 7):
		Dist